In [ ]:
!pip install selenium

In [4]:
!pip install easydict

  Created wheel for easydict: filename=easydict-1.9-py3-none-any.whl size=6350 sha256=a464aaf8ce11556de00ea9a9803455f2e334d91fe705cb1ebea7a1c741e30eb3
  Stored in directory: /Users/hyundukseo/Library/Caches/pip/wheels/b2/1c/36/fd8efc3fa029b1445d869f359ec13e8fbce7fc9e81505bca66
Successfully built easydict


In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from easydict import EasyDict

In [6]:
# example url
# 5개의 item url로 테스트 진행
URL_PATH = 'https://www.musinsa.com/app/goods/1778404?loc=goods_rank'
# URL_PATH = 'https://www.musinsa.com/app/goods/1495284'
# URL_PATH = 'https://www.musinsa.com/app/goods/2018065?loc=goods_rank'
# URL_PATH = 'https://www.musinsa.com/app/goods/885041'
# URL_PATH = 'https://www.musinsa.com/app/goods/2090907/0'

In [12]:
options = webdriver.ChromeOptions()
#chrome_options = webdriver.ChromeOptions()
#chrome_options.add_argument(argument='--headless')
#chrome_options.add_argument(argument='--no-sandbox')
#chrome_options.add_argument(argument='--disable-dev-shm-usage')
# 브라우저를 띄우지 않고 백그라운드에서 크롤링하기 위한 옵션
# options.add_argument('headless') 
# options.add_argument('window-size=1920x1080')
# options.add_argument("disable-gpu")

In [54]:
features = EasyDict()

|locater option | locator |
|---|---|
|id|By.ID|
|class name|By.CLASS_NAME |
|css selector| By.CSS_SELECTOR |
|name| By.NAME |
|link text|By.LINK_TEXT |
|partial link text| By.PARTIAL_LINK_TEXT |
|tag name| By.TAG_NAME |


  
html 상에서 원하는 element를 찾는 함수
```
find_element(locator, dom element location)
```
참고 : https://www.selenium.dev/documentation/webdriver/elements/finders/

In [90]:
def get_rating(driver) :
    try :
        rating = float(driver.find_element(By.CSS_SELECTOR, value="div.estimate-point > p span").text)
    except :
        rating = None
    return rating

def get_likes(driver) :
    try :
        likes = int(driver.find_element(By.CSS_SELECTOR, value="li.product_section_like span").text)
    except :
        likes = None
    return likes
    
def get_cum_sale(driver):
    try:
        cum_sale = int(driver.find_element(By.CSS_SELECTOR, value="li#li_sales_1y strong").text)
    except :
        cum_sale = None
    return cum_sale

def get_buy_age_list(driver):
    buy_age_raw = driver.find_elements(By.CSS_SELECTOR, value="ul.bar_wrap > li span.bar_num")
    buy_age_list = list()
    for buy_age in buy_age_raw :
        buy_age_list.append(int(buy_age.text[:-1]))
    if len(buy_age_list) == 0:
        buy_age_list = None

    return buy_age_list

def get_buy_gender_list(driver): 
    buy_gender_raw = driver.find_elements(By.CSS_SELECTOR, value="dl.label_info > dd")
    buy_gender_list = list()
    for buy_gender in buy_gender_raw :
        buy_gender_list.append(int(buy_gender.text[:-1]))
    if len(buy_gender_list) == 0:
        buy_gender_list = None
    return buy_gender_list

def get_fs_and_fit(driver) :
    guide_all = driver.find_elements(By.CSS_SELECTOR, value="table.table-simple tr")
    four_season_list = list()
    fit_list = list()
    for guide in guide_all :
        t = guide.find_element(By.CSS_SELECTOR, value="th").text
        if t == "계절" :
            seasons = guide.find_elements(By.CSS_SELECTOR, value="td.active")
            for season in seasons :
                four_season_list.append(season.text)
        elif t == "핏" :
            fits = guide.find_elements(By.CSS_SELECTOR, value="td.active")
            for fit in fits :
                fit_list.append(fit.text)
    if len(four_season_list)==0 :
        four_season_list = None
    
    if len(fit_list)==0 :
        fit_list = None
    return four_season_list, fit_list

def get_tags_list(driver) :
    tags_list = list()
    tags_raw = driver.find_elements(By.CSS_SELECTOR, value="li.article-tag-list a.listItem")
    for tag in tags_raw :
        tags_list.append(tag.text[1:])
    return tags_list

def get_four_season(driver) :
    guide_all = driver.find_elements(By.CSS_SELECTOR, value="table.table-simple tr")
    four_season = list()
    for guide in guide_all :
        t = guide.find_element(By.CSS_SELECTOR, value="th").text
        if t == "계절" :
            seasons = guide.find_elements(By.CSS_SELECTOR, value="td.active")
            for season in seasons :
                four_season.append(season.text)
    return four_season



In [91]:
CODI_URL = "https://www.musinsa.com/app/styles/lists"
driver = webdriver.Chrome('./chromedriver',options= options)
driver.get(CODI_URL)
driver.implicitly_wait(2) #페이지를 로딩하는 시간동안 대기

button = driver.find_element(By.CSS_SELECTOR, "button.global-filter__button--mensinsa")
button.click()

container =  driver.find_elements(By.CSS_SELECTOR, "ul.style-list > li")
BASE_CODI_URL = "https://www.musinsa.com/app/styles/views/"
#BASE_ITEM_URL = "https://www.musinsa.com/app/goods/"
for page in container : 
    codi_id = page.find_element(By.CSS_SELECTOR, value = "div.style-list-item__thumbnail > a").get_attribute('onclick')
    codi_id = codi_id.split("'")[1]
    codi_url = BASE_CODI_URL + str(codi_id)
    driver.get(codi_url)
    driver.implicitly_wait(0.5)
    item_urls = driver.find_elements(By.CSS_SELECTOR, "div.box-img > a.styling_img")
    item_urls_list = list()
    for url in item_urls :
        item_url = str(url.get_attribute("href"))
        item_urls_list.append(item_url)

    for url in item_urls_list :
        #item_url = url.get_attribute("href")
        driver.get(url)

        rating = get_rating(driver)
        likes = get_likes(driver)
        cum_sale = get_cum_sale(driver)
        buy_age_list = get_buy_age_list(driver)
        buy_gender_list = get_buy_gender_list(driver)
        four_season_list, fit_list = get_fs_and_fit(driver)
        tags_list = get_tags_list(driver)

        driver.get(codi_url)

        break
    break



driver.close()

/var/folders/my/pb2rr1md6xx7h3qhs4xx52j00000gn/T/ipykernel_63512/2284327775.py:2: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver',options= options)


In [82]:
features

{'rating': 3.5,
 'likes': 54,
 'cum_sales': None,
 'buy_age_list': [23, 11, 14, 33, 4, 15],
 'buy_gender_list': [62, 38],
 'four_season': None,
 'tag': ['반팔', '오버핏', '반팔티셔츠', '그래픽', '오버핏반팔', '파스텔', '로고티셔츠'],
 'fit': None}

In [ ]:
CODI_URL = "https://www.musinsa.com/app/styles/lists"
driver = webdriver.Chrome('./chromedriver',options= options)
driver.get(CODI_URL)
driver.implicitly_wait(2) #페이지를 로딩하는 시간동안 대기

button = driver.find_element(By.CSS_SELECTOR, "button.global-filter__button--mensinsa")
button.click()

container =  driver.find_elements(By.CSS_SELECTOR, "ul.style-list > li")
BASE_CODI_URL = "https://www.musinsa.com/app/styles/views/"
#BASE_ITEM_URL = "https://www.musinsa.com/app/goods/"
codi_urls = list()
for page in container : 
    codi_id = page.find_element(By.CSS_SELECTOR, value = "div.style-list-item__thumbnail > a").get_attribute('onclick')
    codi_id = codi_id.split("'")[1]
    codi_url = BASE_CODI_URL + str(codi_id)
    codi_urls.append(codi_url)

for codi_url in codi_urls :
    driver.get(codi_url)
    driver.implicitly_wait(0.5)

    driver.get(CODI_URL)

In [ ]:
codi_urls

In [85]:
driver = webdriver.Chrome('./chromedriver',options= options)
driver.get("https://www.musinsa.com/app/goods/1110028/0")
four_season = get_four_season(driver)
print(four_season)
driver.close()

/var/folders/my/pb2rr1md6xx7h3qhs4xx52j00000gn/T/ipykernel_63512/3591701411.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome('./chromedriver',options= options)


['봄', '가을']


In [25]:
all_url

[<selenium.webdriver.remote.webelement.WebElement (session="90f40a0a847136d15d5fd7aafc65b188", element="60a47617-6066-48f9-b32a-194e1f194c66")>,
 <selenium.webdriver.remote.webelement.WebElement (session="90f40a0a847136d15d5fd7aafc65b188", element="4f213aa0-f164-4e81-afd4-4cb844e3bec2")>,
 <selenium.webdriver.remote.webelement.WebElement (session="90f40a0a847136d15d5fd7aafc65b188", element="6130624f-54c3-4738-aa24-a09e2868cb0a")>,
 <selenium.webdriver.remote.webelement.WebElement (session="90f40a0a847136d15d5fd7aafc65b188", element="40b95e5f-7ffd-4218-b352-b785e6a1c65c")>,
 <selenium.webdriver.remote.webelement.WebElement (session="90f40a0a847136d15d5fd7aafc65b188", element="9c9d55a7-8877-4ff4-9175-b1152a201b72")>,
 <selenium.webdriver.remote.webelement.WebElement (session="90f40a0a847136d15d5fd7aafc65b188", element="dd6a6105-5e67-4acf-b2a8-523ec459e538")>,
 <selenium.webdriver.remote.webelement.WebElement (session="90f40a0a847136d15d5fd7aafc65b188", element="36c7ee6e-8920-49ce-8ca9-ac

In [14]:
features.name = driver.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[3]/div[3]/span/em').text
features.bigClass = driver.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[3]/div[3]/div[1]/p/a[1]').text
features.middleClass = driver.find_element(By.XPATH, '//*[@id="page_product_detail"]/div[3]/div[3]/div[1]/p/a[2]').text
# features.gender = driver.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]/span[2]/span[1]').text
features.gender = driver.find_element(By.XPATH,'//*[@id="product_order_info"]/div[1]/ul/li[2]/p[2]').text
features.prices = driver.find_element(By.XPATH, '//*[@id="list_price"]').text
features.cumSales = driver.find_element(By.XPATH, '//*[@id="sales_1y_qty"]').text
features.good = driver.find_element(By.XPATH, '//*[@id="product-top-like"]/p[2]/span').text
features.rating = driver.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[6]/p[2]/a/span[2]').text

try:   
    tag_list = driver.find_element(By.XPATH, '//*[@id="product_order_info"]/div[1]/ul/li[7]').text.split('\n')
    features.tag = tag_list
except:
    features.tag = {}
features

{'name': '2WAY 스웻 후드 집업 (MELANGE GREY)',
 'bigClass': '아우터',
 'middleClass': '후드 집업',
 'gender': '2021 S/S / 남 여',
 'prices': '39,000원',
 'cumSales': '14.5만 개 이상',
 'good': '117,981',
 'rating': '4.8',
 'tag': ['#오버핏', '#후드', '#집업후드', '#트레이닝세트', '#셋업', '#후드집업', '#세트']}

- 상품명 : name
- 대분류 : bigClass
- 중분류 : middleClass
- 성별 : gender
- 가격 : prices
- Tag : tag
- 누적판매 : cumSales
- 좋아요 : good
- 별점 : rating

In [133]:
driver.close() #종료 명령어 수행하지 않을 시 프로세스가 계속 남을 수 있음